# Notebook for processing LArPix datalog hdf5 files
Converts LArPix raw hdf5 files (typically with a signature `datalog*_.h5`) into event display files (`datalog*_evd.h5`). The event display files contain hits collected into events and fit with a simple PCA track fitter.

## Paths
Point ``larpix_geometry_libpath`` to the larpix-geometry install.

Point ``evd_libpath`` to the larpix-v2-testing-scripts/event-display install.

In [1]:
larpix_geometry_libpath = '/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-software/larpix-geometry/'
evd_libpath = '/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-software/larpix-v2-testing-scripts/event-display/'

In [2]:
import os
import sys
import glob
sys.path.append(larpix_geometry_libpath)
sys.path.append(evd_libpath)

## Processing loop configuration
Grabs files from ``raw_datapath`` matching ``raw_filestring`` and outputs files to ``out_datapath`` with a name ``out_filestring``.

If ``skip_existing`` is ``False``, existing output files will be deleted before processing.

In [3]:
raw_datapath = '/global/project/projectdirs/dune/data/larpix/raw_data/raw_20_10_12/500_V_cm/'
raw_filestring = 'datalog*_.h5'

out_datapath = os.path.join(os.environ['SCRATCH'],'evd/')
out_filestring = '{}_evd.h5' # filled with raw file basename

skip_existing = False

## Processing configuration
``processing_args`` contains the arguments to pass along to the ``to_evd_file.py`` script.

In [4]:
processing_args = dict(
    configuration_file='/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-calib/datalog_2020_10_12_16_45_30_PDT_config.json',
    geometry_file='/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-software/larpix-geometry/larpixgeometry/layouts/layout-2.3.0.yaml',
    pedestal_file='/global/project/projectdirs/dune/users/pmadigan/larpix/larpix-calib/datalog_2020_10_12_16_45_30_PDT_pedestal.json',
    buffer_size=38400,
    event_dt=150, # clock ticks
    nhit_cut=2, # minimum number of hits in an event
    max_packets=-1,
    vd=1.648, # drift velocity in mm/us
    clock_period=0.1, # clock period for timestamp in us
    dbscan_eps=14., # parameter used by dbscan to find tracks in mm
    dbscan_min_samples=5 # parameter used by dbscan to find tracks
)

In [5]:
input_files = glob.glob(os.path.join(raw_datapath, raw_filestring))
print('fetched:')
for file in input_files:
    print(os.path.basename(file))

fetched:
datalog_2020_10_12_17_44_33_PDT_.h5
datalog_2020_10_12_16_45_30_PDT_.h5
datalog_2020_10_12_16_24_35_PDT_.h5
datalog_2020_10_12_17_14_10_PDT_.h5
datalog_2020_10_12_18_05_20_PDT_.h5
datalog_2020_10_12_16_49_35_PDT_.h5
datalog_2020_10_12_17_20_05_PDT_.h5
datalog_2020_10_12_17_40_19_PDT_.h5
datalog_2020_10_12_18_31_07_PDT_.h5
datalog_2020_10_12_18_09_31_PDT_.h5


## Processing loop!

In [6]:
import to_evd_file
for in_filename in input_files:
    print()
    processing_args['in_filename'] = in_filename
    processing_args['out_filename'] = os.path.abspath(os.path.join(out_datapath,out_filestring).format(os.path.basename(in_filename.split('_.h5')[0])))
    out_dirname = os.path.dirname(processing_args['out_filename'])
    if not os.path.exists(out_dirname):
        print('making directory',out_dirname,'...')
        os.makedirs(out_dirname)
    if os.path.exists(processing_args['out_filename']) and not skip_existing:
        print('deleting existing file',processing_args['out_filename'],'...')
        os.remove(processing_args['out_filename'])
    elif os.path.exists(processing_args['out_filename']) and skip_existing:
        print('skipping existing file',processing_args['out_filename'],'...')
        continue
        
    # run!
    for key, arg in sorted(processing_args.items()):
        print(key,'\t:\t',arg)
    to_evd_file.main(**processing_args)
    
print('Done!')


deleting existing file /global/cscratch1/sd/pmadigan/evd/datalog_2020_10_12_17_44_33_PDT_evd.h5 ...
buffer_size 	:	 38400
clock_period 	:	 0.1
configuration_file 	:	 /global/project/projectdirs/dune/users/pmadigan/larpix/larpix-calib/datalog_2020_10_12_16_45_30_PDT_config.json
dbscan_eps 	:	 14.0
dbscan_min_samples 	:	 5
event_dt 	:	 150
geometry_file 	:	 /global/project/projectdirs/dune/users/pmadigan/larpix/larpix-software/larpix-geometry/larpixgeometry/layouts/layout-2.3.0.yaml
in_filename 	:	 /global/project/projectdirs/dune/data/larpix/raw_data/raw_20_10_12/500_V_cm/datalog_2020_10_12_17_44_33_PDT_.h5
max_packets 	:	 -1
nhit_cut 	:	 2
out_filename 	:	 /global/cscratch1/sd/pmadigan/evd/datalog_2020_10_12_17_44_33_PDT_evd.h5
pedestal_file 	:	 /global/project/projectdirs/dune/users/pmadigan/larpix/larpix-calib/datalog_2020_10_12_16_45_30_PDT_pedestal.json
vd 	:	 1.648
packets parsed: 1211609	events found: 55655...Done!
finishing up...
done

deleting existing file /global/cscratch1/s

# Event display
Use this to view files that were generated.

In [7]:
import quick_display
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

In [8]:
%matplotlib widget
geom = [-155.15,155.15,-155.15,155.15,0,150,4.434,50]
output_files = sorted([os.path.basename(path) for path in glob.glob(out_datapath+'/*.h5')])
fig = None
@widgets.interact
def display(event_id=widgets.IntText(value=0, description='Index:'),
           filename=widgets.Dropdown(options=output_files, description="File"),
           nhit_sel=widgets.IntText(value=0, description='NHit cut:')):
    f = quick_display.open_file(os.path.join(out_datapath,filename))
    if len(f['events']):
        mask = f['events']['nhit'] > nhit_sel
        if abs(event_id) < np.sum(mask):
            quick_display.generate_plots(f['events'][mask][event_id], f, geom)
        else:
            print('reached end of file')
    else:
        print('no events in file')

interactive(children=(IntText(value=0, description='Index:'), Dropdown(description='File', options=('datalog_2…